## Solution

In [1]:
import requests
import json
import pandas as pd

In [2]:
item     = ""
url      = "https://cat-fact.herokuapp.com"
endpoint = "/facts/random?animal_type="
animals  = ["dog", "cat"]

results = {
    "DocID"      : [],
    "Text"       : [],
    "AnimalType" : [],
    "UpdateDate" : []
}

target_counts = 5 # It takes some time, in the solutin I'll use only 5 records

In [3]:
# Iter in each animal
for item in animals:
    print("Retrieving information about {}".format(item))
    # 1 .. 100 to get some facts about animals
    index = 1 # Index will be reset for each animal

    while index <= target_counts:
        print("\tFact of a {} nº {}".format(item, index), end = "")
        
        # API response
        fact = requests.get(str(url+endpoint+item)).json()
        
        if not fact["status"].get("verified"):
            # If is not verified response retry
            print(" Response not verified, retrying...")
        else:
            
            # If response it's verified, get all the required information
            print(" Verified response of user {}".format(fact["user"])) 
            
            # Get information of document ID
            results["DocID"].append(fact["_id"])
            # Get information of Text of each fact
            results["Text"].append(fact["text"])
            # Get information about animal type
            results["AnimalType"].append(fact["type"])
            # Get information about updated date
            results["UpdateDate"].append(fact["updatedAt"])
            
            index = index + 1

Retrieving information about dog
	Fact of a dog nº 1 Response not verified, retrying...
	Fact of a dog nº 1 Response not verified, retrying...
	Fact of a dog nº 1 Response not verified, retrying...
	Fact of a dog nº 1 Verified response of user 5a9ac18c7478810ea6c06381
	Fact of a dog nº 2 Response not verified, retrying...
	Fact of a dog nº 2 Response not verified, retrying...
	Fact of a dog nº 2 Response not verified, retrying...
	Fact of a dog nº 2 Response not verified, retrying...
	Fact of a dog nº 2 Verified response of user 5a9ac18c7478810ea6c06381
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...
	Fact of a dog nº 3 Response not verified, retrying...


In [4]:
# Show the final dataframe
df = pd.DataFrame(results)
df

,DocID,Text,AnimalType,UpdateDate
0,5b7a025b2048fd0014e9d873,Puppies and kittens can be adopted as early as...,dog,2020-08-23T20:20:01.611Z
1,5c871eb6357f5a0eeaac122d,"Dogs’ eyes contain a special membrane, called ...",dog,2020-09-06T20:20:01.882Z
2,5c871f27357f5a0eeaac1233,45% of dogs sleep in their owner’s bed.,dog,2020-09-17T20:20:02.088Z
3,5b7a031a2048fd0014e9d882,The Basenji is the only barkless dog in the wo...,dog,2020-08-23T20:20:01.611Z
4,5c871dac85c8b40e7ed683e7,"It’s rumored that, at the end of the Beatles s...",dog,2020-09-02T20:20:04.107Z
5,591f98783b90f7150a19c199,Cats sleep 16 to 18 hours per day. When cats a...,cat,2020-08-23T20:20:01.611Z
6,591f98783b90f7150a19c1d6,The smallest breed of domestic cat is the Sing...,cat,2020-08-23T20:20:01.611Z
7,591f97e88dec2e14e3c20b05,"In the wild, lions live for an average of 12 y...",cat,2020-08-23T20:20:01.611Z
8,591f98703b90f7150a19c17d,"Stroking a cat can help to relieve stress, and...",cat,2020-08-23T20:20:01.611Z
9,5d9c556168a764001553b382,A cat has 244 bones in its entire body—even mo...,cat,2020-08-23T20:20:01.611Z
